In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'svg'

## Import module

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import os
import numpy as np
from torchsummary import summary
from torchvision import transforms                                                                                                                                        
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600
from PIL import Image
import pandas as pd

import sys
sys.path.append("../../../src/")
sys.path.append("../../")
import model
from datasets import imagenet
import config
from aux.utils import obtain_features_map, load_imgs, zscore, extract_valid
from aux.visualization import visualize_features_map
from utils.visualizations.visualize import concat_imgs, preprocess_arrays

## Main

In [3]:
def main(exp, selected_layer, selected_filter, class_index=950):
    num_class = 30
    class_index = class_index
    print("=> class index: {}".format(class_index))
    backbone = "vgg16"
    resume = "037-0"
    color_map = "jet"
    model_dir = "/home/lincolnzjx/Desktop/Interpretation/saved/models"
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    ## Load modal from dict
    
    # model
    net = model.Network(backbone=backbone, num_classes=1000, 
                        selected_layer=selected_layer)
    net.to(device)
    
    # resume from model
    resume_exp = resume.split("-")[0]
    resume_epoch = resume.split("-")[1]
    print("Resume from model from exp: {} at epoch {}".format(resume_exp, resume_epoch))
    resume_path = os.path.join(model_dir, str(resume_exp), str(resume_epoch))
    ckpt = torch.load(resume_path, map_location=device)
    net.load_state_dict(ckpt, strict=False) 
    
    summary(net, (3, 224, 224))
    
    ## Load Original Data 
    
    # Load data
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    reverse_mean = [-0.485, -0.456, -0.406]
    reverse_std = [1/0.229, 1/0.224, 1/0.225]
    train_transform = None
    train_transform = transforms.Compose([                                                                                                                                
        transforms.Resize((224, 224), interpolation=Image.BILINEAR),                                                                                              
        transforms.ToTensor(),                                                                                                                                            
        #transforms.Normalize(mean, std)                                                                                                                                   
           ])         
    trainset = imagenet.ImageNet(root="/media/lincolnzjx/HardDisk/Datasets/", 
                                 is_train=True, transform=train_transform)
    
    trainset.set_data([class_index], num_class)
    imgs_path = []                                                                                                                                                            
    images = []
    labels = []
    for img, label, img_path in trainset:                                                                                                                                     
        images.append(img.unsqueeze(0))                                                                                                                                       
        labels.append(label)                                                                                                                                                  
        imgs_path.append(img_path)  
    
    ## Create some need path
    
    save_dir = "../../../saved/pack/"
    ################### Hyper-Parameter #######################
    # exp = resume_exp
    # epoch = resume_epoch
    ##########################################################
    ab_path = os.path.join(save_dir, exp)
    
    index2image = {index: item.split("/")[-1].split(".")[0] for index, item in enumerate(imgs_path)}
    index2image
    
    images_cpu = np.array([image.detach().clone().cpu().numpy().squeeze() for image in images])
    
    # Load image
    # ext may be different.
    optimized_data, valid_imgs_path, valid_imgs_index = load_imgs(ab_path, imgs_path, non_exists_ok=True, ext=".png")
    valid_imgs, valid_labels = extract_valid(images, labels, valid_imgs_index)
    optimized_data_zscore = zscore(optimized_data, mean, std)
    images_zscore = zscore(images_cpu, mean, std)
    
    # Move to device
    opt_image = torch.from_numpy(optimized_data_zscore).to(device)
    original_image = torch.from_numpy(images_zscore).to(device)
    
    ## Obtain feature from conv layers.
    
    ori_activation_maps = net.get_activation_maps(original_image, 
                                                  selected_layer=selected_layer)[0]
    opt_activation_maps = net.get_activation_maps(opt_image, 
                                                  selected_layer=selected_layer)[0]
    
    images_cpu_p, optimized_data_cpu, ori_activation_maps_cpu, \
       opt_activation_maps_cpu, optimized_data_cpu_scale, \
       diff_activation_maps_cpu,\
       ori_activation_maps_cpu_n1e1, opt_activation_maps_cpu_n1e1, \
       diff_activation_maps_cpu_n1e1,\
       ori_activation_maps_cpu_n2e1, opt_activation_maps_cpu_n2e1, \
       diff_activation_maps_cpu_n2e1,\
       ori_activation_maps_cpu_n4e1, opt_activation_maps_cpu_n4e1,\
       diff_activation_maps_cpu_n4e1 = \
       preprocess_arrays(images_cpu, optimized_data,
                         ori_activation_maps, opt_activation_maps,
                         selected_filter, color_map=color_map)
    concated_imgs, concated_fms, concated_imgs_fms, out_not_resize_img = \
      concat_imgs(
        images_cpu_p, optimized_data_cpu,
        ori_activation_maps_cpu, opt_activation_maps_cpu,
        optimized_data_cpu_scale, diff_activation_maps_cpu,
        ori_activation_maps_cpu_n1e1, opt_activation_maps_cpu_n1e1,
        diff_activation_maps_cpu_n1e1,
        ori_activation_maps_cpu_n2e1, opt_activation_maps_cpu_n2e1,
        diff_activation_maps_cpu_n2e1,
        ori_activation_maps_cpu_n4e1, opt_activation_maps_cpu_n4e1,
        diff_activation_maps_cpu_n4e1)
    # imgs
    save_imgs_path = os.path.join("../../../saved/pack/", exp, 
                                  "concat.png")
    concated_imgs.save(save_imgs_path)
    # fms 
    save_fms_path = os.path.join("../../../saved/pack/", exp, 
                                  "concat_fms.png")
    concated_fms.save(save_fms_path)
    # imgs & fms
    save_imgs_fms_path = os.path.join("../../../saved/pack/", exp, 
                                      "concat_imgs_fms.png")
    concated_imgs_fms.save(save_imgs_fms_path)
    # not_resize
    save_imgs_not_resize_path = os.path.join("../../../saved/pack/", exp,
                                             "concat_imgs_fm_not_resize.png")
    out_not_resize_img.save(save_imgs_not_resize_path)

## PATH

In [4]:
# excel_path = "Single.xlsx"
excel_path = "Batches.xlsx"
sheet = "950"
# sheet = "950"
excepts = ["052148"]

## RUN

In [5]:
xlsx = pd.ExcelFile(excel_path)
excel = pd.read_excel(xlsx, sheet, Sdtype={"exp": str, 
                                           "selected_layer": int,
                                           "selected_filter": int, 
                                           "class_index": int}).values
for data in excel:
    exp, selected_layer, selected_filter, class_index = data[0], data[1], \
        data[2], data[3]
    exp = exp.replace("x", "")
    if exp in excepts:
        print("=> Skip")
    else:
        print("=> RUN ", exp, selected_layer, selected_filter, class_index)
        main(exp, selected_layer, selected_filter, class_index)

=> RUN  061300 1 16 950
=> class index: 950
Only Keep 1th layers before.
Resume from model from exp: 037 at epoch 0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
             VGG16-3         [-1, 64, 224, 224]               0
Total params: 1,792
Trainable params: 1,792
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 73.50
Params size (MB): 0.01
Estimated Total Size (MB): 74.08
----------------------------------------------------------------
Len of new dataset is :30
../../../saved/pack/061300
Load from ../../../saved/pack/061300/46785/n07747607_10061.png
Load from ../../../saved/pack/061300/46785/n07747607_10068.png
Load from ../../../saved/pack/061300/46785/n07747607_10110.png
Load f

=> Register fhook 1
=> Register fhook 1
pixel min is not 0.
=> RUN  7004 3 28 950
=> class index: 950
Only Keep 3th layers before.
Resume from model from exp: 037 at epoch 0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
             VGG16-5         [-1, 64, 224, 224]               0
Total params: 38,720
Trainable params: 38,720
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 122.50
Params size (MB): 0.15
Estimated Total Size (MB): 123.22
----------------------------------------------------------------
Len of new dataset is :30
../../../saved/pack/7004
Load fr

=> Register fhook 6
MAX: ori smaller.
MAX: ori smaller.
MAX: ori smaller.
MAX: ori smaller.
MAX: ori smaller.
MAX: ori smaller.
MAX: ori smaller.
MAX: ori smaller.
MAX: ori smaller.
MAX: ori smaller.
MAX: ori smaller.
MAX: ori smaller.
=> RUN  7006 8 99 950
=> class index: 950
Only Keep 8th layers before.
Resume from model from exp: 037 at epoch 0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
         

Len of new dataset is :30
../../../saved/pack/70081
Load from ../../../saved/pack/70081/376321/n07747607_10061.png
Load from ../../../saved/pack/70081/376321/n07747607_10068.png
Load from ../../../saved/pack/70081/376321/n07747607_10110.png
Load from ../../../saved/pack/70081/376321/n07747607_10134.png
Load from ../../../saved/pack/70081/376321/n07747607_10162.png
Load from ../../../saved/pack/70081/376321/n07747607_10166.png
Load from ../../../saved/pack/70081/376321/n07747607_10178.png
Load from ../../../saved/pack/70081/376321/n07747607_10179.png
Load from ../../../saved/pack/70081/376321/n07747607_10184.png
Load from ../../../saved/pack/70081/376321/n07747607_10192.png
Load from ../../../saved/pack/70081/376321/n07747607_10204.png
Load from ../../../saved/pack/70081/376321/n07747607_10205.png
Load from ../../../saved/pack/70081/376321/n07747607_1021.png
Load from ../../../saved/pack/70081/376321/n07747607_10213.png
Load from ../../../saved/pack/70081/376321/n07747607_10217.png
Load

=> Register fhook 18
=> Register fhook 18
MAX: ori smaller.
MAX: ori smaller.
=> RUN  060493 18 101 950
=> class index: 950
Only Keep 18th layers before.
Resume from model from exp: 037 at epoch 0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1,

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

=> Register fhook 25
=> Register fhook 25
=> RUN  060495 27 161 950
=> class index: 950
Only Keep 27th layers before.
Resume from model from exp: 037 at epoch 0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
      

=> Register fhook 29
=> Register fhook 29
